<a href="https://colab.research.google.com/github/daniel-szoverfi/BiblAI/blob/main/pkp_lab1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Labor 1

# Bevezetés a CUDA GPU programozásba

## Mi a párhuzamos programozás?

A párhuzamos programozás olyan programozási paradigma, ahol több számítási feladat egyidejűleg, egymással párhuzamosan fut. Hagyományos, szekvenciális programozás esetén a program utasításai egymás után, lépésről lépésre hajtódnak végre. Ezzel szemben a párhuzamos programozás lehetővé teszi, hogy több utasítás egyidejűleg fusson.

## Miért van szükség párhuzamos programozásra?

A processzorteljesítmény növelése az évek során elérte fizikai korlátait (órajel, hőtermelés). Az egymagos processzorok teljesítménye nem növelhető a végtelenségig, így többmagos processzorokat kezdtek gyártani. Bizonyos feladatok természetüknél fogva párhuzamosíthatók (pl. képfeldolgozás, mátrixműveletek, szimulációk), amelyek hatékonyan felgyorsíthatók párhuzamos feldolgozással.

## GPU-k és a CUDA szerepe

A grafikus processzorok (GPU-k) eredetileg a számítógépes grafikához készültek, de felismerték, hogy bizonyos számítási feladatok (különösen azok, amelyek sok adaton ugyanazt a műveletet végzik) sokkal gyorsabban futhatnak rajtuk. A GPU-k több ezer egyszerűbb számítási magot tartalmaznak, szemben a CPU-k néhány komplex magjával.

A [GPGPU](https://www.gigabyte.com/Glossary/gpgpu) rövidítés a "General Purpose Graphics Processing Unit"-ra utal, ami egy olyan számítási technológia, amely során a grafikus kártyák (GPU-k) számítási kapacitását használjuk általános célú számítások végrehajtására. Mint láttuk, a GPU-k eredetileg kifejezetten grafikus feladatok hatékony megoldására lettek tervezve, mint például a képfeldolgozás vagy 3D grafika. Azonban, az elmúlt években a GPU-k egyre inkább beépültek az általános célú számítások világába is.

A GPGPU technológia alkalmazása lehetővé teszi, hogy a GPU-k nagy számítási kapacitását felhasználják olyan feladatok elvégzésére, mint például a gépi tanulás, a kriptográfiai műveletek, a tudományos szimulációk vagy a nagy adathalmazok feldolgozása. A GPGPU technológia használata jelentősen felgyorsíthatja ezeket a számítási feladatokat, és csökkentheti azok elvégzésének idejét.

Az NVIDIA 2007-ben bemutatta a CUDA (Compute Unified Device Architecture) platformot, amely lehetővé teszi a programozók számára, hogy a GPU-kat általános célú számításokra is használhassák (GPGPU - General-Purpose computing on Graphics Processing Units).

## Heterogén párhuzamos programozás

A CUDA programozást heterogén párhuzamos programozásnak is nevezik, mert:

1. **Heterogén rendszer**: A CPU (host) és a GPU (device) együttműködve oldják meg a feladatot
2. **Különböző architektúrák**: A CPU és a GPU eltérő architektúrával és memóriarendszerrel rendelkezik
3. **Eltérő feladatkörök**: A CPU általában a szekvenciális részeket, a GPU a párhuzamosítható részeket hajtja végre



## CUDA Runtime API


A [CUDA Runtime API](https://docs.nvidia.com/cuda/cuda-runtime-api/index.html) egy függvénykönyvtárból és a C++ szintaxis egyszerű kiegészítéséből áll.

A CUDA programokat `*.cu` (nem `*.c` vagy `*.cpp`) kiterjesztésű állományokban írjuk. Mint meglátjuk, alapvetően C++ kódot írünk, melyet néha kiegészíthetünk gyorsítón futtatandó programrészletekkel, úgynevezett "CUDA kernelekkel".

Az alábbi kódrészlet egy tökéletesen működő CUDA program, csak éppenséggel még nem tartalmaz gyorsító specifikus kódot.

```cpp
#include <iostream>

int main(int argc, char** argv)
{
    std::cout << "Hello World!" << std::endl;
    return 0;
}
```


Ha a programunkban szeretnénk kihasználni a GPU számítási kapacitását lehetőségeit is, akkor GPU kódot is írnunk kell. Ez a számításokhoz szükséges adatokat előkészíti, átmásolja, meghívja a GPU kernelt, majd az adatokat visszamásolja. A CUDA forráskódok általában keverten tartalmazzák a CPU, és GPU kódokat.

A GPU-n futó számítást úgynevezett a fentebb említett kernel függvények megadásával tudjuk megvalósítani. Ezek a `__global__` előtaggal rendelkező függvények.

### Kompilálás és futtatás

A CUDA (`*.cu` kiterjesztésű) programjaink kompilálásához, használjuk a következő parancsot:  

```
!nvcc mysurcefile.cu -o programname
```

majd sikeres kompilálás esetében, futtatáshoz:
```
!./programname
```

## CUDA program általános szerkezete

A CUDA programok speciális felépítéssel rendelkeznek, ami a heterogén rendszer sajátosságaiból adódik. Az alábbiakban megnézzük a tipikus CUDA program szerkezetét és a főbb lépéseket.

## 1. Program inicializálás és feladatmeghatározás

Minden CUDA program a host (CPU) oldalon kezdődik, ahol:
- Meghatározzuk a feladat méretét (pl. feldolgozandó elemek száma)
- Inicializáljuk a bemeneti adatokat
- Kiszámoljuk a memória igényeket

```c
int n = 1000000;  // Elemek száma
size_t size = n * sizeof(float);  // Szükséges memória mérete bájtokban
```

## 2. Memória kezelés a heterogén rendszerben

Kulcsfontosságú megérteni, hogy a CPU és GPU külön memóriaterülettel rendelkezik, így explicit memóriakezelésre van szükség:

### a) Host memória foglalás (CPU oldal)
```c
float *h_input = (float*)malloc(size);
float *h_output = (float*)malloc(size);

// Adatok inicializálása
for (int i = 0; i < n; i++) {
    h_input[i] = rand() / (float)RAND_MAX;
}
```

### b) Device memória foglalás (GPU oldal)
```c
float *d_input = NULL;
float *d_output = NULL;
cudaMalloc((void**)&d_input, size);
cudaMalloc((void**)&d_output, size);
```

### c) Adatok másolása host-ról device-ra
```c
cudaMemcpy(d_input, h_input, size, cudaMemcpyHostToDevice);
```

## 3. Kernel definíció

A kernel a GPU-n futó függvény, amelyet a `__global__` kulcsszóval jelölünk:

```c
__global__ void processData(float *input, float *output, int n) {
    // Egyedi szálazonosító számítása
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    
    // Ellenőrzés, hogy a szál érvényes adatelemen dolgozik-e
    if (idx < n) {
        // Tényleges munka elvégzése (példa: érték duplázása)
        output[idx] = 2.0f * input[idx];
    }
}
```

## 4. Végrehajtási konfiguráció és kernel indítás

A kernel indításakor meg kell határozni a végrehajtási konfigurációt (hogyan szerveződnek a szálak blokkokba, a blokkok egy rácsba (grid)):

```c
// Blokkméret meghatározása (szálak száma blokkonként)
int threadsPerBlock = 256;

// Blokkok számának kiszámítása
// Felfelé kerekítünk, hogy minden adatelemhez jusson szál
int blocksPerGrid = (n + threadsPerBlock - 1) / threadsPerBlock;

// Kernel indítása a megadott konfigurációval
processData<<<blocksPerGrid, threadsPerBlock>>>(d_input, d_output, n);

// Aszinkron végrehajtás - megvárjuk a befejezést
cudaDeviceSynchronize();
```

## 5. Eredmények visszaolvasása

A feldolgozás után az eredményeket vissza kell másolni a GPU-ról a CPU memóriába:

```c
cudaMemcpy(h_output, d_output, size, cudaMemcpyDeviceToHost);
```

## 6. Eredmények feldolgozása és erőforrások felszabadítása

A CPU oldalon feldolgozzuk az eredményeket, majd felszabadítjuk a lefoglalt erőforrásokat:

```c
// Eredmények feldolgozása (pl. ellenőrzés)
for (int i = 0; i < 10; i++) {
    printf("Input: %f, Output: %f\n", h_input[i], h_output[i]);
}

// Erőforrások felszabadítása
cudaFree(d_input);
cudaFree(d_output);
free(h_input);
free(h_output);
```

## 7. Hibakezelés (jó gyakorlat)

A CUDA API hívások hibakódot adnak vissza, amit érdemes ellenőrizni:

```c
cudaError_t err = cudaMemcpy(d_input, h_input, size, cudaMemcpyHostToDevice);
if (err != cudaSuccess) {
    printf("CUDA error: %s\n", cudaGetErrorString(err));
    // Hibakezelés...
}
```




## Szálazonosító számítása a CUDA-ban

### Egyedi szálazonosítók működése

A CUDA architektúrában minden szálnak szüksége van egy egyedi azonosítóra, hogy tudja, melyik adatelemen kell dolgoznia. Ez az azonosító kiszámítása kulcsfontosságú a párhuzamos végrehajtás során. A CUDA hierarchikus modellben gondolkodik: a szálak blokkokba vannak szervezve, a blokkok pedig rácsot (grid) alkotnak.

Az egyedi globális szálazonosító (global thread ID) kiszámítása a következő képlettel történik 1D szervezás esetében:

```
globalId = blockIdx.x * blockDim.x + threadIdx.x
```

Ahol:
- `blockIdx.x`: Az aktuális blokk indexe a rácsban
- `blockDim.x`: A blokkon belüli szálak száma (szálak per blokk)
- `threadIdx.x`: A szál indexe a blokkon belül

### Példák az 1D azonosító számításra

- Ha 2 blokkunk van, egyenként 3 szállal:
  - 0. blokk, 0. szál: `0 * 3 + 0 = 0`
  - 0. blokk, 1. szál: `0 * 3 + 1 = 1`
  - 0. blokk, 2. szál: `0 * 3 + 2 = 2`
  - 1. blokk, 0. szál: `1 * 3 + 0 = 3`
  - 1. blokk, 1. szál: `1 * 3 + 1 = 4`
  - 1. blokk, 2. szál: `1 * 3 + 2 = 5`


Ez a számítási modell biztosítja, hogy minden szál pontosan tudja, melyik adatelemen kell dolgoznia, függetlenül attól, hogy hány blokkot és szálat használunk a párhuzamos végrehajtáshoz.


Késobbiekben majd megltájuk, hogy például kétdimenziós (pl. mátrixműveleteknél) szervezés esetében is tudunk mindig linearizálni, egyedi azonosítot számítani:

```c
int row = blockIdx.y * blockDim.y + threadIdx.y;
int col = blockIdx.x * blockDim.x + threadIdx.x;
int idx = row * width + col;  // Linearizált index
```


## Példa: Teljes CUDA program a vektorok összeadására

Két vektor elemeinek összegzése párhuzamosan CUDA-val:

```c
#include <stdio.h>
#include <cuda_runtime.h>

// GPU kernel - minden szál egy elemet dolgoz fel
__global__ void vectorAdd(int *a, int *b, int *c, int n) {
    int i = blockDim.x * blockIdx.x + threadIdx.x;
    if (i < n) {
        c[i] = a[i] + b[i];
    }
}

int main() {
    int n = 1000;
    int size = n * sizeof(int);
    
    // Host memória foglalása
    int *h_a = (int *)malloc(size);
    int *h_b = (int *)malloc(size);
    int *h_c = (int *)malloc(size);
    
    // Vektorok inicializálása
    for (int i = 0; i < n; i++) {
        h_a[i] = i;
        h_b[i] = i;
    }
    
    // Device memória foglalása
    int *d_a, *d_b, *d_c;
    cudaMalloc(&d_a, size);
    cudaMalloc(&d_b, size);
    cudaMalloc(&d_c, size);
    
    // Adatok másolása a host-ról a device-ra
    cudaMemcpy(d_a, h_a, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, h_b, size, cudaMemcpyHostToDevice);
    
    // Kernel indítása 256 szállal blokkonként
    int threadsPerBlock = 256;
    int blocksPerGrid = (n + threadsPerBlock - 1) / threadsPerBlock;
    vectorAdd<<<blocksPerGrid, threadsPerBlock>>>(d_a, d_b, d_c, n);
    
    // Eredmény visszamásolása a device-ról a host-ra
    cudaMemcpy(h_c, d_c, size, cudaMemcpyDeviceToHost);
    
    // Eredmény ellenőrzése
    for (int i = 0; i < 10; i++) {
        printf("%d + %d = %d\n", h_a[i], h_b[i], h_c[i]);
    }
    
    // Memória felszabadítása
    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);
    free(h_a);
    free(h_b);
    free(h_c);
    
    return 0;
}
```

A CUDA programozás alapelvei a példában fellelhetőek:
1. Szétválasztjuk a kódot CPU (host) és GPU (device) részekre
2. A GPU kódot speciális `__global__` kulcsszóval jelöljük (kernel)
3. Memóriát foglalunk mindkét eszközön, és explicit másolással kommunikálunk
4. Meghatározzuk a párhuzamosítás szintjét (hány blokk és hány szál/blokk)

A CUDA előnye, hogy C nyelvi környezetben programozhatunk, minimális nyelvi kiterjesztésekkel, miközben kihasználjuk a GPU-k párhuzamos feldolgozási képességeit.

## Google Colaboratory

A Google Colaboratory (röviden Colab) egy ingyenes online platform, amely lehetővé teszi a felhasználók számára a (főleg) Python programozási nyelv interaktív környezetében történő munkavégzését. Az egyik fő előnye, hogy ingyenes GPU és TPU (Tensor Processing Unit) számítási erőforrásokat biztosít a felhasználók számára, azzal a cállal, hogy lehetővé tegye a nagyobb adatméretű és bonyolultabb gépi tanulási és adatelemzési feladatok végrehajtását.

Az GPU erőforrások igénybevétele érdekében, a felhasználóknak a futásidejű környezet típusát kell áttálítsa. Ehhez, a `Runtime/Change runtime` menűpontban válasszuk ki, hogy `GPU`.

A `GPU` elérhetőségét teszteln tudjuk a következő kóddal:

In [ ]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


A Nvidia kompájler verziójának lekérése:

In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


A kód cellákkban csak Python kód futtatható direkt modon. Ezért, [%%writefile magic](https://ipython.readthedocs.io/en/stable/interactive/magics.html) parancsal fogjuk kiírni a lemezre a programjainkat, majd ezeket más kódcellákban kompiláljuk és futtatjuk.

Például, az alábbi példaprogram GPU-n végzi el két szám összeadását.

In [ ]:
%%writefile vecadd.cu
#include <cuda.h>
#include <stdio.h>

__global__ void add(int a, int b, int* c)
{
    *c = a + b;
    return;
}

int main(int argc, char** argv)
{
    int c;
    int* dev_c;

	  //memória foglalás a GPU-n
    cudaMalloc((void**)&dev_c, sizeof(int) );

    add<<<1,1>>>(1, 2, dev_c);

    cudaMemcpy(&c, dev_c, sizeof(int), cudaMemcpyDeviceToHost);

    printf("a + b = %d\n", c);

    //lefolglat memória felszabadítása
    cudaFree(dev_c);
    return 0;
}

Writing vecadd.cu


És íme meg is jelenik a `vecadd.cu` állomány:

In [ ]:
!ls

sample_data  vecadd.cu


Kompilálás:

In [ ]:
!nvcc vecadd.cu

Ellenörizzük ha megjelent a futtatható bináris program (`a.out`):

In [ ]:
!ls

a.out  sample_data  vecadd.cu


Futtatás:

In [ ]:
!./a.out

a + b = 0


# Feladatok

1. Írjunk egy CUDA programot, mely a [`cudaError_t cudaGetDeviceCount (int * count)`](https://docs.nvidia.com/cuda/cuda-runtime-api/group__CUDART__DEVICE.html#group__CUDART__DEVICE_1g18808e54893cfcaafefeab31a73cc55f), [`cudaError_t cudaGetDeviceProperties (struct cudaDeviceProp * prop, int device )`](https://docs.nvidia.com/cuda/cuda-runtime-api/group__CUDART__DEVICE.html#group__CUDART__DEVICE_1g1bf9d625a931d657e08db2b4391170f0) függvények segítségével, kiírja a CUDA kompatibilis GPU főbb paramétereit.
A jellemzőket tartalmazó `cudaDeviceProp` struktúra definíciója a következő:

  ```cpp
  struct cudaDeviceProp {
                char name[256];
                cudaUUID_t uuid;
                size_t totalGlobalMem;
                size_t sharedMemPerBlock;
                int regsPerBlock;
                int warpSize;
                size_t memPitch;
                int maxThreadsPerBlock;
                int maxThreadsDim[3];
                int maxGridSize[3];
                int clockRate;
                size_t totalConstMem;
                int major;
                int minor;
                size_t textureAlignment;
                size_t texturePitchAlignment;
                int deviceOverlap;
                int multiProcessorCount;
                int kernelExecTimeoutEnabled;
                int integrated;
                int canMapHostMemory;
                int computeMode;
                int maxTexture1D;
                int maxTexture1DMipmap;
                int maxTexture1DLinear;
                int maxTexture2D[2];
                int maxTexture2DMipmap[2];
                int maxTexture2DLinear[3];
                int maxTexture2DGather[2];
                int maxTexture3D[3];
                int maxTexture3DAlt[3];
                int maxTextureCubemap;
                int maxTexture1DLayered[2];
                int maxTexture2DLayered[3];
                int maxTextureCubemapLayered[2];
                int maxSurface1D;
                int maxSurface2D[2];
                int maxSurface3D[3];
                int maxSurface1DLayered[2];
                int maxSurface2DLayered[3];
                int maxSurfaceCubemap;
                int maxSurfaceCubemapLayered[2];
                size_t surfaceAlignment;
                int concurrentKernels;
                int ECCEnabled;
                int pciBusID;
                int pciDeviceID;
                int pciDomainID;
                int tccDriver;
                int asyncEngineCount;
                int unifiedAddressing;
                int memoryClockRate;
                int memoryBusWidth;
                int l2CacheSize;
                int persistingL2CacheMaxSize;
                int maxThreadsPerMultiProcessor;
                int streamPrioritiesSupported;
                int globalL1CacheSupported;
                int localL1CacheSupported;
                size_t sharedMemPerMultiprocessor;
                int regsPerMultiprocessor;
                int managedMemory;
                int isMultiGpuBoard;
                int multiGpuBoardGroupID;
                int singleToDoublePrecisionPerfRatio;
                int pageableMemoryAccess;
                int concurrentManagedAccess;
                int computePreemptionSupported;
                int canUseHostPointerForRegisteredMem;
                int cooperativeLaunch;
                int cooperativeMultiDeviceLaunch;
                int pageableMemoryAccessUsesHostPageTables;
                int directManagedMemAccessFromHost;
                int accessPolicyMaxWindowSize;
            }
   ```

2. Írjunk egy "Hello, World" CUDA kernelt, melyben minden szál kiírja az [egyedi azonosítóját](https://blog.usejournal.com/cuda-thread-indexing-fb9910cba084). Hívjuk meg a kernelt különböző rács és blokk konfigurációkkal. Használjuk a [cudaError_t cudaDeviceSynchronize ( void )](https://docs.nvidia.com/cuda/cuda-runtime-api/group__CUDART__DEVICE.html#group__CUDART__DEVICE_1g10e20b05a95f638a4071a655503df25d) függvényt a kernel hívás bevárására.

3. Futtasuk a két tömb összeadása példát.
4. Készítsünk CUDA programot, amely egy vektort megszoroz egy konstans értékkel, majd megkeresi a vektor legnagyobb értékét.

  **Specifikáció**:
  - Hozzunk létre egy 100000 elemű, véletlenszámokat tartalmazó vektort
  - A konstans szorzó értéke legyen 2.5
  - Írjunk kernelt a vektor konstanssal való szorzásához
  - A maximumot egyelőre a CPU oldalon keressük meg a szorzás után

  **Lépések**:
  1. Hozzunk létre és inicializáljunk egy vektort a host (CPU) oldalon
  2. Foglaljunk memóriát a device (GPU) oldalon
  3. Másoljuk az adatokat a host-ról a device-ra
  4. Implementáljunk és indítsunk egy kernelt a vektor szorzásához
  5. Másoljuk vissza az eredményt a device-ról a host-ra
  6. Keressük meg a vektor legnagyobb értékét a CPU-n
  7. Írjuk ki az eredményt és szabadítsuk fel az erőforrásokat




In [ ]:
%%writefile device_props.cu
#include <iostream>
#include <cuda_runtime.h>

int main() {
    int deviceCount;
    cudaError_t err = cudaGetDeviceCount(&deviceCount);

    if (err != cudaSuccess) {
        std::cerr << "Hiba a GPU-k lekérdezésekor: " << cudaGetErrorString(err) << std::endl;
        return 1;
    }

    std::cout << "CUDA kompatibilis GPU-k száma: " << deviceCount << std::endl;

    for (int i = 0; i < deviceCount; i++) {
        cudaDeviceProp prop;
        cudaGetDeviceProperties(&prop, i);

        std::cout << "\nGPU #" << i << ": " << prop.name << std::endl;
        std::cout << "   CUDA Compute Capability: " << prop.major << "." << prop.minor << std::endl;
        std::cout << "   Összes globális memória: " << prop.totalGlobalMem / (1024 * 1024) << " MB" << std::endl;
        std::cout << "   Osztott memória blokk/SM: " << prop.sharedMemPerBlock / 1024 << " KB" << std::endl;
        std::cout << "   Regiszterek blokk/SM: " << prop.regsPerBlock << std::endl;
        std::cout << "   Warp méret: " << prop.warpSize << std::endl;
        std::cout << "   Max. szál/blokk: " << prop.maxThreadsPerBlock << std::endl;
        std::cout << "   Max. blokk dimenziók: [" << prop.maxThreadsDim[0] << ", " << prop.maxThreadsDim[1] << ", " << prop.maxThreadsDim[2] << "]" << std::endl;
        std::cout << "   Max. grid méret: [" << prop.maxGridSize[0] << ", " << prop.maxGridSize[1] << ", " << prop.maxGridSize[2] << "]" << std::endl;
        std::cout << "   Órajel: " << prop.clockRate / 1000 << " MHz" << std::endl;
        std::cout << "   Multiprocesszorok száma: " << prop.multiProcessorCount << std::endl;
        std::cout << "   PCI bus ID: " << prop.pciBusID << std::endl;
        std::cout << "   PCI device ID: " << prop.pciDeviceID << std::endl;
    }

    return 0;
}


Writing device_props.cu


In [ ]:
!nvcc -arch=sm_75 device_props.cu

In [ ]:
!./a.out

CUDA kompatibilis GPU-k száma: 1

GPU #0: Tesla T4
   CUDA Compute Capability: 7.5
   Összes globális memória: 15095 MB
   Osztott memória blokk/SM: 48 KB
   Regiszterek blokk/SM: 65536
   Warp méret: 32
   Max. szál/blokk: 1024
   Max. blokk dimenziók: [1024, 1024, 64]
   Max. grid méret: [2147483647, 65535, 65535]
   Órajel: 1590 MHz
   Multiprocesszorok száma: 40
   PCI bus ID: 0
   PCI device ID: 4


In [ ]:
%%writefile hello_world_kernel.cu
#include <iostream>
#include <cuda_runtime.h>

__global__ void hello_world_kernel() {
    int threadId = blockIdx.x * blockDim.x + threadIdx.x;
    printf("Hello from thread %d\n", threadId);
}

void launch_kernel(int num_blocks, int num_threads_per_block) {
    std::cout << "Konfiguráció: " << num_blocks << " blokk, " << num_threads_per_block << " szál/blokk" << std::endl;
    hello_world_kernel<<<num_blocks, num_threads_per_block>>>();
    cudaError_t err = cudaGetLastError();
    if (err != cudaSuccess) {
        std::cerr << "CUDA error: " << cudaGetErrorString(err) << std::endl;
    }
    cudaDeviceSynchronize();
}

int main() {
    launch_kernel(1, 1);
    launch_kernel(2, 4);
    launch_kernel(4, 16);

    return 0;
}


Overwriting hello_world_kernel.cu


In [ ]:
!nvcc -arch=sm_75 hello_world_kernel.cu

In [ ]:
!./a.out

Konfiguráció: 1 blokk, 1 szál/blokk
Hello from thread 0
Konfiguráció: 2 blokk, 4 szál/blokk
Hello from thread 0
Hello from thread 1
Hello from thread 2
Hello from thread 3
Hello from thread 4
Hello from thread 5
Hello from thread 6
Hello from thread 7
Konfiguráció: 4 blokk, 16 szál/blokk
Hello from thread 0
Hello from thread 1
Hello from thread 2
Hello from thread 3
Hello from thread 4
Hello from thread 5
Hello from thread 6
Hello from thread 7
Hello from thread 8
Hello from thread 9
Hello from thread 10
Hello from thread 11
Hello from thread 12
Hello from thread 13
Hello from thread 14
Hello from thread 15
Hello from thread 16
Hello from thread 17
Hello from thread 18
Hello from thread 19
Hello from thread 20
Hello from thread 21
Hello from thread 22
Hello from thread 23
Hello from thread 24
Hello from thread 25
Hello from thread 26
Hello from thread 27
Hello from thread 28
Hello from thread 29
Hello from thread 30
Hello from thread 31
Hello from thread 32
Hello from thread 33
Hello f

*Vektor szorzása konstansal*

In [ ]:
%%writefile vektor_osszeadas.cu
# include <stdio.h>
# include <cuda_runtime.h>

// GPU kernel - minden szál egy elemet dolgoz fel
__global__ void vectorAdd(int *a, int *b, int *c, int n) {
    int i = blockDim.x * blockIdx.x + threadIdx.x;
    if (i < n) {
        c[i] = a[i] + b[i];
    }
}

int main() {
    int n = 1000;
    int size = n * sizeof(int);

    // Host memória foglalása
    int *h_a = (int *)malloc(size);
    int *h_b = (int *)malloc(size);
    int *h_c = (int *)malloc(size);

    // Vektorok inicializálása
    for (int i = 0; i < n; i++) {
        h_a[i] = i;
        h_b[i] = i;
    }

    // Device memória foglalása
    int *d_a, *d_b, *d_c;
    cudaMalloc(&d_a, size);
    cudaMalloc(&d_b, size);
    cudaMalloc(&d_c, size);

    // Adatok másolása a host-ról a device-ra
    cudaMemcpy(d_a, h_a, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, h_b, size, cudaMemcpyHostToDevice);

    // Kernel indítása 256 szállal blokkonként
    int threadsPerBlock = 256;
    int blocksPerGrid = (n + threadsPerBlock - 1) / threadsPerBlock;
    vectorAdd<<<blocksPerGrid, threadsPerBlock>>>(d_a, d_b, d_c, n);

    // Eredmény visszamásolása a device-ról a host-ra
    cudaMemcpy(h_c, d_c, size, cudaMemcpyDeviceToHost);

    // Eredmény ellenőrzése
    for (int i = 0; i < 10; i++) {
        printf("%d + %d = %d\n", h_a[i], h_b[i], h_c[i]);
    }

    // Memória felszabadítása
    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);
    free(h_a);
    free(h_b);
    free(h_c);

    return 0;
}

Writing vektor_osszeadas.cu


In [ ]:
!nvcc -arch=sm_75 vektor_osszeadas.cu

In [ ]:
!./a.out

0 + 0 = 0
1 + 1 = 2
2 + 2 = 4
3 + 3 = 6
4 + 4 = 8
5 + 5 = 10
6 + 6 = 12
7 + 7 = 14
8 + 8 = 16
9 + 9 = 18


In [ ]:
%%writefile vektor_konstans_szorzas.cu
#include <iostream>
#include <cuda_runtime.h>
#include <cstdlib>
#include <ctime>

__global__ void vector_multiply_with_constant(float* a, float* b, float k, int n) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    if (i < n) {
        b[i] = a[i] * k;
    }
}

int main () {
  int n = 100000;
  int size = n * sizeof(int);
  float k = 2.5;

  // Host memria foglalása
  float *h_a = (float *)malloc(size);
  float *h_b = (float *)malloc(size);

  srand(time(0));

  for (int i = 0; i < n; i++) {
    h_a[i] = rand() / (float)RAND_MAX;
  }

  // Device memoria foglalása
  float *d_a, *d_b;
  cudaMalloc(&d_a, size);
  cudaMalloc(&d_b, size);

  // Adatok masolása a host-ról a device-ra
  cudaMemcpy(d_a, h_a, size, cudaMemcpyHostToDevice);

  int threadsPerBlock = 256;
  int blocksPerGrid = (n + threadsPerBlock - 1) / threadsPerBlock;
  vector_multiply_with_constant<<<blocksPerGrid, threadsPerBlock>>>(d_a, d_b, k, n);

  // Copy back from device to host
  cudaMemcpy(h_b, d_b, size, cudaMemcpyDeviceToHost);

   float max_val = h_b[0];
  for(int i = 1; i < n; i++) {
      if (h_b[i] > max_val) {
          max_val = h_b[i];
      }
  }

  std::cout << "Max value: " << max_val << std::endl;

    // Free allocated memory
    cudaFree(d_a);
    cudaFree(d_b);
    free(h_a);
    free(h_b);

  return 0;



}


Overwriting vektor_konstans_szorzas.cu


In [ ]:
!nvcc -arch=sm_75 vektor_konstans_szorzas.cu

In [ ]:
!./a.out

Max value: 2.49999
